In [1]:
import torch
import argparse
import time
import os
import sys
import random
import numpy as np
import math
import os, json, copy
import pandas as pd
import torch
import sklearn
import pandas as pd
from torch.utils.data import DataLoader,TensorDataset
from torch.utils.data import Subset
import os, json, copy
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pandas.core.frame import DataFrame
from torch.utils.tensorboard import SummaryWriter
import random
import time, math
import logging
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn.functional as f
import torch.nn.functional as F

configs={
  "mode": "train_mixed",
  "seed": 11,
  "batch_size": 256,
  "device": "cuda",
  "colab": False,
  "num_workers": 3,
  "lr": 0.001,
  "weight_decay": 0.0005,
  "epochs": 50,
  "lr_decay": 5,
  "lr_decay_rate": 0.8,
  "custom_loss": "fbeta_loss",
  "preprocess": True,
  "split_dataset": False,
  "lambda": 0.3,
  "beta": 15.0,
  "split_ratio":0.8,
  "only_train":True,
}
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda and configs['device'] =='cuda' else "cpu")
configs['device']=str(device)
time_data = time.strftime('%m-%d_%H-%M-%S', time.localtime(time.time()))
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pandas.core.frame import DataFrame

def fix_seed(seed):
    #Seed 고정
    random.seed(seed)
    torch.manual_seed(seed)
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [2]:
### Fix Seed ###

random.seed(configs['seed'])
torch.manual_seed(configs['seed'])
torch.random.manual_seed(configs['seed'])
torch.cuda.manual_seed_all(configs['seed'])
np.random.seed(configs['seed'])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
### data path designation ###
current_path=os.getcwd()
if configs['colab']==True:
    par_path=os.path.abspath(os.path.join(current_path, os.pardir))
    current_path=os.path.join(par_path,'drive','MyDrive')
data_path=os.path.join(current_path)
save_path=os.path.join(current_path,'training_data')
if os.path.exists(save_path) == False:
    os.mkdir(save_path)
if os.path.exists(os.path.join(save_path,time_data)) == False:
    os.mkdir(os.path.join(save_path,time_data))

In [4]:
## save configuration ##
def load_params(configs,current_path, file_name):
    ''' replay_name from file_name '''
    with open(os.path.join(current_path, 'training_data', '{}.json'.format(file_name)), 'r') as fp:
        configs = json.load(fp)
    return configs


def save_params(configs,current_path, time_data):
    with open(os.path.join(current_path, 'training_data', '{}.json'.format(time_data)), 'w') as fp:
        json.dump(configs, fp, indent=2)

save_params(configs,current_path,time_data)
########################

## logger ##
def set_logging_defaults(time_data,logdir):
    # set basic configuration for logging
    logging.basicConfig(format="[%(asctime)s] [%(name)s] %(message)s",
                        level=logging.INFO,
                        handlers=[logging.FileHandler(os.path.join(logdir,time_data, 'log.txt')),
                                  logging.StreamHandler(os.sys.stdout)])

    # log cmdline argumetns
    logger = logging.getLogger('main')
    logger.info(' '.join(os.sys.argv))

set_logging_defaults(time_data,save_path)
logger = logging.getLogger('main')
############

## calculation of the score ##
def calc_score(predictions, targets,score_dict):
    predictions=predictions.detach().clone().cpu()
    targets=targets.detach().clone().cpu()
    accuracy = accuracy_score(targets, predictions,)
    if targets.unique().size(0)==3:
        avg='macro'
    elif targets.unique().size(0)==2:
        avg='binary'
    precision = precision_score(targets, predictions,average=avg,zero_division=0)
    recall = recall_score(targets, predictions,average=avg,zero_division=0)
    f1 = f1_score(targets, predictions,average=avg,zero_division=0)
    score_dict['total']+=targets.size(0)
    score_dict['accuracy'] = accuracy*100.0
    score_dict['precision'] = precision*100.0
    score_dict['recall'] = recall*100.0
    score_dict['f1score']=f1*100.0
    return score_dict

[2021-07-21 12:18:07,566] [main] C:\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py -f C:\Users\Medical IT\AppData\Roaming\jupyter\runtime\kernel-f23957ab-f8d7-4ce5-bd75-446f07f26e54.json


In [5]:
# train.csv의 column을 학습에서 feature로 사용허가 위해서 preprocessing을 진행 할 때에, 각 고유분류코드의 우범 및 핵심우범을 저지를 기댓값에 따라 가중치를 주기 위해
# 기댓값을 계산하여 Json file로 저장하는 Class입니다.
# 추가적으로, train.csv의 컬럼 데이터 이외에 컬럼 데이터를 조합하거나 가공해서 만든 새로운 데이터 컬럼에 대해서도 계산을 진행합니다.

class MappingJsonGenerator():
    def __init__(self, train_csv, fillna_str, drop_list, only_train, data_path = None):
        
        self.only_train = only_train
        self.train_csv=train_csv
        self.fillna_str = fillna_str
        self.drop_list = drop_list
        
        self.train_csv['HS_upper'] = self.train_csv['HS10단위부호'] // 100000000
        self.train_csv['HS_middle'] = self.train_csv['HS10단위부호'] // 1000000
        self.train_csv['HS_low'] = self.train_csv['HS10단위부호'] // 10000

        self.train_csv['관세율구분코드_1자리']=self.train_csv['관세율구분코드'].str.slice(start = 0, stop = 1)

        self.train_csv['단위무게(KG)가격'] = (self.train_csv['과세가격원화금액']/self.train_csv['신고중량(KG)']).map(lambda x: np.round(x, 0)).map(str)

        self.train_csv.drop(['과세가격원화금액', '신고중량(KG)'], axis = 1,errors='ignore',inplace=True)
        
        if self.only_train:
            # train_indices = np.load('./data/custom_contest/train_indices.npy')
            train_indices = np.load(os.path.join(data_path,'train_indices.npy'))
            self.train_valid_csv = self.train_csv
            self.train_csv = self.train_valid_csv[self.train_valid_csv.columns].iloc[train_indices]
            self.train_valid_csv=self.train_valid_csv.drop(['우범여부', '핵심적발'] + self.drop_list, axis = 1,errors='ignore')
            self.train_valid_csv = self.train_valid_csv.fillna(self.fillna_str)


        self.crime = np.array(self.train_csv['우범여부'])
        self.priority = np.array(self.train_csv['핵심적발'])
        self.train_csv=self.train_csv.drop(['우범여부', '핵심적발'] + self.drop_list, axis = 1,errors='ignore')

        self.train_csv = self.train_csv.fillna(self.fillna_str)
        self.column_list = np.array(self.train_csv.columns, dtype=str)

        self.crime_idx = np.where(self.crime == 1)[0]
        self.non_priority_idx = np.where(self.priority == 1)[0]
        self.priority_idx = np.where(self.priority == 2)[0]
        self.dictionary = dict()

        self.crime_threshold_no_valid = [0]*21
        self.priority_threshold_no_valid = [5]*21
        
    def __call__(self):
        return self.forward()
    
    def forward(self):
        train_np = np.array(self.train_csv, dtype = str)
        print(train_np.shape)
        if self.only_train:
            train_valid_np = np.array(self.train_valid_csv, dtype = str)
            print('calc count_ratio for only train')
            print(train_valid_np.shape)
        else:
            train_valid_np = train_np
            print('calc count_ratio for train + valid')
            print(train_np.shape)

        for i, col in enumerate(self.column_list):
            crime_count_mean, crime_ratio_mean = 0., 0.
            priority_count_mean, priority_ratio_mean = 0., 0.
            self.dictionary[col] = {}
            concat = np.concatenate([train_valid_np[:, i]], axis = 0)
            total_code, total_count = np.unique(concat, return_counts=True)
            crime_code, crime_count = np.unique(train_np[:, i][self.crime_idx], return_counts=True)

            _, total_priority_count = np.unique(np.concatenate((train_np[:, i][self.priority_idx],train_np[:, i][self.non_priority_idx]),axis=0), return_counts=True)
            priority_code, priority_count = np.unique(train_np[:, i][self.priority_idx], return_counts=True)

            crime_ratio = np.empty((total_count.shape[0], ))
            
            priority_ratio = np.empty((total_count.shape[0], ))

            c_idx = 0
            p_idx = 0
            for assign_idx, c in enumerate(total_code) : 
                self.dictionary[col][c] = {}
                self.dictionary[col][c]['total_count'] = int(total_count[assign_idx])

                if c not in crime_code : 
                    crime_ratio[assign_idx] = 0.
                    self.dictionary[col][c]['crime_count'] = int(0)
                    self.dictionary[col][c]['is_crime_mask'] = True
                else :
                    crime_ratio[assign_idx] = np.round(crime_count[c_idx] / total_count[assign_idx], 4)
                    self.dictionary[col][c]['crime_count'] = int(crime_count[c_idx])
                    crime_count_mean += float(crime_count[c_idx])
                    crime_ratio_mean += float(crime_ratio[assign_idx]*crime_count[c_idx])
                    if crime_count[c_idx] <= self.crime_threshold_no_valid[i]:
                        self.dictionary[col][c]['is_crime_mask'] = True
                    else : 
                        self.dictionary[col][c]['is_crime_mask'] = False
                    c_idx += 1

                self.dictionary[col][c]['crime_ratio'] = float(crime_ratio[assign_idx])
                self.dictionary[col][c]['onehot'] = int(assign_idx+1)
            
            crime_concat = np.concatenate([crime_ratio.reshape(-1, 1), total_code.reshape(-1, 1)], axis = 1)
            crime_concat = np.array(sorted(crime_concat, key = lambda x : x[0], reverse=True))

            for k, c in enumerate(crime_concat[:, 1]):
                self.dictionary[col][c]['sorted_crime_onehot'] = int(k+1)


                
            for assign_idx,p in enumerate(total_code):
                if p not in priority_code: 
                    priority_ratio[assign_idx] = 0.
                    self.dictionary[col][p]['priority_count'] = int(0)
                    self.dictionary[col][p]['is_priority_mask'] = True
                else :
                    assign_priority_idx=np.where(priority_code==p)[0]
                    #priority_ratio[assign_idx] = np.round(priority_count[p_idx] / total_priority_count[assign_priority_idx], 4)
                    priority_ratio[assign_idx] = np.round(priority_count[p_idx] / self.dictionary[col][p]['crime_count'], 4)
                    self.dictionary[col][p]['priority_count'] = int(priority_count[p_idx])
                    priority_count_mean += float(priority_count[p_idx])
                    priority_ratio_mean += float(priority_ratio[assign_idx]*priority_count[p_idx])
                    if priority_count[p_idx] <= self.priority_threshold_no_valid[i]:
                        self.dictionary[col][p]['is_priority_mask'] = True
                    else : 
                        self.dictionary[col][p]['is_priority_mask'] = False
                    p_idx += 1

                
                self.dictionary[col][p]['priority_ratio'] = float(priority_ratio[assign_idx])

            priority_concat = np.concatenate([priority_ratio.reshape(-1, 1), total_code.reshape(-1, 1)], axis = 1)
            priority_concat = np.array(sorted(priority_concat, key = lambda x : x[0], reverse=True))

            for k, c in enumerate(priority_concat[:, 1]):
                self.dictionary[col][c]['sorted_priority_onehot'] = int(k+1)

            self.dictionary[col]['crime_count_mean'] = crime_count_mean / train_valid_np.shape[0]
            self.dictionary[col]['crime_ratio_mean'] =crime_ratio_mean / train_valid_np.shape[0]
            self.dictionary[col]['priority_count_mean'] = priority_count_mean / train_valid_np.shape[0]
            self.dictionary[col]['priority_ratio_mean'] = priority_ratio_mean / train_valid_np.shape[0]

        return self.dictionary

In [6]:
# 위에서 설명드린 Class를 이용하여 Json file을 생성하고 해당 정보를 이용하여 preprocessing을 진행하는 Class입니다.
# 우범여부를 학습하기 위한 데이터에 사용되는 _crime_transform과 핵심적발을 학습하기 위한 데이터에 사용되는 _priority_transform이 주요 동작을 수행합니다.
# 추가적으로, train.csv의 컬럼 데이터 이외에 컬럼 데이터를 조합하거나 가공해서 만든 새로운 데이터 컬럼에 대해서도 계산을 진행합니다.

def find_digits(x):
    temp = math.log2(x)
    if temp == math.floor(temp):
        n = temp
    else:
        n = math.floor(temp) + 1
    return int(n)


def binary_transform(x):
    if x == 0:
        return '0'
    else:
        binary=''
        while x>0:
            x, mod = divmod(x,2)
            binary += str(mod)
        return binary


class RescaleNumeric:
    def __init__(self):
        self.minmax_scale=preprocessing.MinMaxScaler(feature_range=(0,1))
    def __call__(self,x):
        x_scale=self.minmax_scale.fit_transform(np.log(x+1).reshape(-1,1))
        return x_scale


class Preprocessing:
    def __init__(self,data_path,configs):
        self.configs=configs
        self.data_path=data_path
        # load mapping dictionary
        train_dataframe,test_dataframe=self._load_dataset()
        npy_dict=dict()
        npy_dict['crime_targets']=train_dataframe.pop('우범여부')
        npy_dict['priority_targets']=train_dataframe.pop('핵심적발')
        npy_dict['train_indices'], npy_dict['valid_indices']=self._split_indices(train_dataframe,npy_dict['priority_targets'])
        for key in npy_dict.keys():
            if isinstance(npy_dict,DataFrame):
                npy_dict[key]=npy_dict[key].to_numpy()
            np.save(os.path.join(self.data_path,'{}.npy'.format(key)),npy_dict[key])
        del npy_dict
        train_dataframe,test_dataframe=self._load_dataset()
        self.mapping_dict = MappingJsonGenerator(train_dataframe,'Missing', ['신고번호', '신고일자', '관세율','검사결과코드'], self.configs['only_train'], data_path)() #'과세가격원화금액', '신고중량(KG)',
        print("Generate Json complete")
        # save
        with open(os.path.join(data_path,'mapping.json'), 'w') as fp:
            json.dump(self.mapping_dict, fp, indent=2)
        # load
        with open(os.path.join(data_path,'mapping.json'), 'r') as fp:
            self.mapping_dict = copy.deepcopy(json.load(fp))

    def _load_dataset(self):
        train_dataframe=copy.deepcopy(pd.read_csv(os.path.join(self.data_path,'train.csv')))
        test_dataframe=copy.deepcopy(pd.read_csv(os.path.join(self.data_path,'test.csv')))
        return train_dataframe,test_dataframe

    def run(self) ->dict:
        npy_dict={}
        #load dataset
        train_dataframe,test_dataframe=self._load_dataset()
        if self.configs['mode']=='record':
            data_frame_list=[test_dataframe]
            data_type_list=['test']
        else:
            data_frame_list=[train_dataframe,test_dataframe]
            data_type_list=['train','test']
        #transform and save the dataset
        for csv_dataframe,data_type in zip(data_frame_list,data_type_list):
            npy_dict['{}_priority_data'.format(data_type)]=self._priority_transform(copy.deepcopy(csv_dataframe))
            npy_dict['{}_crime_data'.format(data_type)]=self._crime_transform(copy.deepcopy(csv_dataframe))
        for key in npy_dict.keys():
            if isinstance(npy_dict,DataFrame):
                npy_dict[key]=npy_dict[key].to_numpy()
            np.save(os.path.join(self.data_path,'{}.npy'.format(key)),npy_dict[key])
        return npy_dict

    def _split_indices(self,dataframe:DataFrame,targets:np.ndarray)->np.ndarray:
        indices=np.arange(len(dataframe))
        train_indices,valid_indices=train_test_split(indices,stratify=targets,random_state=self.configs['seed'],test_size=1-self.configs['split_ratio'],train_size=self.configs['split_ratio'])
        return train_indices, valid_indices

    def _crime_transform(self, dataframe:DataFrame)->np.ndarray:
        rescaler=RescaleNumeric()
        """
        categorical_features = ['통관지세관부호', '신고인부호', '수입자부호', '해외거래처부호', '특송업체부호', 
                                '수입통관계획코드', '수입신고구분코드', '수입거래구분코드', '수입종류코드', 
                                '징수형태코드', '운송수단유형코드', '반입보세구역부호', 'HS_upper', 'HS_middle', 
                                '적출국가코드', '원산지국가코드', '관세율구분코드']
        """
        categorical_features = self.mapping_dict.keys()
        dataframe['HS_upper'] = dataframe['HS10단위부호'] // 100000000 # 위 2자리
        dataframe['HS_middle'] = dataframe['HS10단위부호'] // 1000000 # 위 4자리
        dataframe['HS_low'] = dataframe['HS10단위부호'] // 10000 # 위 4자리
        dataframe['관세율구분코드_1자리']=dataframe['관세율구분코드'].str.slice(start = 0, stop = 1)
        dataframe['단위무게(KG)가격'] = (dataframe['과세가격원화금액']/dataframe['신고중량(KG)']).map(lambda x: np.round(x, 0)).map(str)
        dataframe.drop('HS10단위부호',axis=1,inplace=True)
        dataframe['세금'] = (dataframe['과세가격원화금액']*dataframe['관세율']/dataframe['신고중량(KG)']/100.0)
        numeric_features = ['신고중량(KG)', '과세가격원화금액','관세율']
        dataframe.fillna('Missing', inplace=True)
        for column in numeric_features:
            dataframe[column] = rescaler(dataframe.pop(column).to_numpy())
        np_data = dataframe[['신고중량(KG)', '과세가격원화금액','관세율']].to_numpy()
        dataframe.drop(['신고일자','신고번호','우범여부','핵심적발' ,'수입자부호'],axis=1,inplace=True,errors='ignore')#,'HS10단위부호'
        len_df = len(dataframe.index)
        
        add_count_ratio_list=['crime_count','priority_count','total_count']
        reg_count_ratio_list=['crime_count','priority_count','total_count']
        np_encoding=None
        print("Before crime transform shape",dataframe.shape)
        for i,column in enumerate(categorical_features):
            if column not in dataframe.columns:
                continue
            dataframe[column] = dataframe[column].map(str)
            dict_col = self.mapping_dict[column]
            np_count_ratio = np.zeros((len_df,len(add_count_ratio_list)))
            if column in ['HS_upper','특송업체부호']:
                np_ratio_addition = np.zeros((len_df,1))
                for row in dataframe[column].index: 
                    val_data = dataframe[column][row]
                    if val_data not in dict_col.keys():
                        continue
                    else:
                        np_ratio_addition[row]=dict_col[val_data]['crime_ratio']
                np_count_ratio=np.concatenate((np_count_ratio,np_ratio_addition),axis=1)
            for row in dataframe[column].index: 
                val_data = dataframe[column][row]
                # value you want to add
                for idx, add_instance in enumerate(add_count_ratio_list):
                    if val_data not in dict_col.keys():
                        if add_instance in ['crime_ratio','priority_ratio'] : np_count_ratio[row][idx] = 0.0
                        else : np_count_ratio[row][idx] = 0
                    else :
                        if (not dict_col[val_data]['is_crime_mask']) and (dict_col[val_data]['crime_ratio'] >= 0.3) :
                            if add_instance != 'total_count':
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]*1.5
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]*1.5
                            else :
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                        elif (not dict_col[val_data]['is_crime_mask']) and (dict_col[val_data]['crime_ratio'] < 0.3) :
                            np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                            np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                        elif (dict_col[val_data]['is_crime_mask']) :
                            if add_instance != 'total_count':
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]*0.5
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]*0.5
                            else :
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                        else :
                            np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                            np_count_ratio[row][idx] = dict_col[val_data][add_instance]

            # regularization
            for idx,reg_instance in enumerate(reg_count_ratio_list):
                np_count_ratio[:,idx] = (np_count_ratio[:,idx]-np_count_ratio[:,idx].mean())/(np_count_ratio[:,idx].var())
            
            if np_encoding is None: # 이진화 안하는 경우 ->위 주석처리 
                np_data = np.concatenate((np_data,np_count_ratio, ), axis=1)
            else: #이진화 하는 경우 -> 위 주석처리 x
                np_encoding = np_encoding[:,::-1]
                np_data = np.concatenate((np_data,np_count_ratio,np_encoding ), axis=1)

            print('\r[{}/{}] Finished Process'.format(i+1,len(categorical_features)),end='')
                
        print("After crime transform shape",np_data.shape)
        return np_data

    def _priority_transform(self, dataframe:DataFrame)->np.ndarray:
        rescaler=RescaleNumeric()
        """
        categorical_features = ['통관지세관부호', '신고인부호', '수입자부호', '해외거래처부호', '특송업체부호', 
                                '수입통관계획코드', '수입신고구분코드', '수입거래구분코드', '수입종류코드', 
                                '징수형태코드', '운송수단유형코드', '반입보세구역부호', 'HS_upper', 'HS_middle', 
                                '적출국가코드', '원산지국가코드', '관세율구분코드']
        """
        dataframe.fillna('Missing', inplace=True)
        categorical_features = self.mapping_dict.keys()
        dataframe['HS_upper'] = dataframe['HS10단위부호'] // 100000000 # 위 2자리
        dataframe['HS_middle'] = dataframe['HS10단위부호'] // 1000000 # 위 4자리
        dataframe['HS_low'] = dataframe['HS10단위부호'] // 10000 # 위 4자리
        dataframe.drop('HS10단위부호',axis=1,inplace=True)
        dataframe['관세율구분코드_1자리']=dataframe['관세율구분코드'].str.slice(start = 0, stop = 1)
        dataframe['단위무게(KG)가격'] = (dataframe['과세가격원화금액']/dataframe['신고중량(KG)']).map(lambda x: np.round(x, 0)).map(str)
        dataframe['세금'] = (dataframe['과세가격원화금액']*dataframe['관세율']/dataframe['신고중량(KG)']/100.0)
        numeric_features = ['신고중량(KG)', '과세가격원화금액','관세율']
        # numeric_features = ['신고중량(KG)', '과세가격원화금액','관세율']
        for column in numeric_features:
            dataframe[column] = rescaler(dataframe.pop(column).to_numpy())
        np_data = dataframe[['신고중량(KG)', '과세가격원화금액','관세율']].to_numpy()
        np_encoding=None
        dataframe.drop(['신고일자','신고번호','우범여부','핵심적발', '수입자부호'],axis=1,inplace=True,errors='ignore')#,'HS10단위부호'
        len_df = len(dataframe.index)
        add_count_ratio_list=['priority_count','crime_count']
        reg_count_ratio_list=['priority_count','crime_count']

        print("Before priority transform shape",dataframe.shape)
        for i,column in enumerate(categorical_features):
            if column not in dataframe.columns:
                continue
            dataframe[column] = dataframe[column].map(str)
            dict_col = self.mapping_dict[column]
            np_count_ratio = np.zeros((len_df,len(add_count_ratio_list)))
            if column in []:
                np_ratio_addition = np.zeros((len_df,1))
                for row in dataframe[column].index: 
                    val_data = dataframe[column][row]
                    if val_data not in dict_col.keys():
                        continue
                    else:
                        np_ratio_addition[row]=dict_col[val_data]['priority_ratio']
                np_count_ratio=np.concatenate((np_count_ratio,np_ratio_addition),axis=1)
            for row in dataframe[column].index: 
                val_data = dataframe[column][row]

                # value you want to add
                for idx, add_instance in enumerate(add_count_ratio_list):
                    if val_data not in dict_col.keys():
                        if add_instance in ['crime_ratio','priority_ratio'] : np_count_ratio[row][idx] = 0.0
                        else : np_count_ratio[row][idx] = 0
                    else :
                        if (not dict_col[val_data]['is_priority_mask']) and (dict_col[val_data]['priority_ratio'] >= 0.3) :
                            if add_instance != 'crime_count':
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]*1.5
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]*1.5
                            else :
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                        elif (not dict_col[val_data]['is_priority_mask']) and (dict_col[val_data]['priority_ratio'] < 0.3) :
                            np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                            np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                        elif (dict_col[val_data]['is_priority_mask']) :
                            if add_instance != 'crime_count':
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]*0.5
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]*0.5
                            else :
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                                np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                        else :
                            np_count_ratio[row][idx] = dict_col[val_data][add_instance]
                            np_count_ratio[row][idx] = dict_col[val_data][add_instance]

            # regularization
            for idx,reg_instance in enumerate(reg_count_ratio_list):
                np_count_ratio[:,idx] = (np_count_ratio[:,idx]-np_count_ratio[:,idx].mean())/(np_count_ratio[:,idx].var())
            
            if np_encoding is None: # 이진화 안하는 경우 ->위 주석처리 
                np_data = np.concatenate((np_data,np_count_ratio, ), axis=1)
            else: #이진화 하는 경우 -> 위 주석처리 x
                np_encoding = np_encoding[:,::-1]
                np_data = np.concatenate((np_data,np_count_ratio,np_encoding ), axis=1)

            print('\r[{}/{}] Finished Process'.format(i+1,len(categorical_features)),end='')
                
        print("After priority transform shape",np_data.shape)
        return np_data
        

In [7]:
# 앞선 preprocessing이 완료된 numpy type의 데이터를 load하고, 학습에 이용되는 DataLoader를 생성하는 함수입니다.

def load_dataset(data_path,configs):
    # get data
    if configs['preprocess']==True:
        # npy_dict=get_data(data_path,configs)
        preprocessing=Preprocessing(data_path,configs)
        npy_dict=preprocessing.run()
    if 'sj' in configs['mode']:
        train_data=np.load(os.path.join(data_path,'sj_train_data.npy'))
        train_targets=np.load(os.path.join(data_path,'sj_train_targets.npy'))
        valid_data=np.load(os.path.join(data_path,'sj_valid_data.npy'))
        valid_targets=np.load(os.path.join(data_path,'sj_valid_targets.npy'))

        train_dataset=TensorDataset(train_data,train_targets)
        valid_dataset=TensorDataset(valid_data,valid_targets)
        return train_dataset,valid_dataset
    else:    
        npy_dict={
            'train_crime_data':np.load(os.path.join(data_path,'train_crime_data.npy')),
            'train_priority_data':np.load(os.path.join(data_path,'train_priority_data.npy')),
            'crime_targets':np.load(os.path.join(data_path,'crime_targets.npy')), # y_1 (0,1)
            'priority_targets':np.load(os.path.join(data_path,'priority_targets.npy')), #y_2 (0,1,2)
            'train_indices':np.load(os.path.join(data_path,'train_indices.npy')),
            'valid_indices':np.load(os.path.join(data_path,'valid_indices.npy')),
            'test_crime_data':np.load(os.path.join(data_path,'test_crime_data.npy')),
            'test_priority_data':np.load(os.path.join(data_path,'test_priority_data.npy')),
        }
    
    if configs['mode'] in ['record','tsne_mixed','tsne_priority','tsne_crime']:
        return npy_dict
    #type casting
    if 'xgboost' not in configs['mode']:
        for key in npy_dict.keys():
            npy_dict[key]=torch.from_numpy(npy_dict[key])
            if npy_dict[key].dtype in [torch.float64,torch.double]:
                npy_dict[key]=npy_dict[key].float()

    #data separation
        if 'crime' in configs['mode']:
            crime_dataset=TensorDataset(npy_dict['train_crime_data'],npy_dict['crime_targets'])
            #crime
            train_dataset=Subset(crime_dataset,npy_dict['train_indices'])
            valid_dataset=Subset(crime_dataset,npy_dict['valid_indices'])

        elif 'priority' in configs['mode']:
            #priority
            train_data=npy_dict['train_priority_data'][npy_dict['train_indices'].long()]
            valid_data=npy_dict['train_priority_data'][npy_dict['valid_indices'].long()]

            train_target=npy_dict['priority_targets'][npy_dict['train_indices'].long()]
            valid_target=npy_dict['priority_targets'][npy_dict['valid_indices'].long()]

            # select 1 and 2
            train_data=train_data[torch.logical_or(train_target==1,train_target==2)]
            valid_data=valid_data[torch.logical_or(valid_target==1,valid_target==2)]
            train_target=train_target[torch.logical_or(train_target==1,train_target==2)]
            valid_target=valid_target[torch.logical_or(valid_target==1,valid_target==2)]

            # change target 1 to 0, 2 to 1
            train_target[train_target==1]=0
            train_target[train_target==2]=1
            valid_target[valid_target==1]=0
            valid_target[valid_target==2]=1
            train_dataset=TensorDataset(train_data, train_target)
            valid_dataset=TensorDataset(valid_data, valid_target)

        elif configs['mode']=='train_mixed':
            #mixed
            mixed_dataset=TensorDataset(npy_dict['train_crime_data'],npy_dict['train_priority_data'],npy_dict['crime_targets'],npy_dict['priority_targets'])
            train_dataset=Subset(mixed_dataset,npy_dict['train_indices'])
            valid_dataset=Subset(mixed_dataset,npy_dict['valid_indices'])

        else:
            print('No dataset')
            raise NotImplementedError
        return train_dataset, valid_dataset

    else: #XGBOOST
        if 'crime' in configs['mode']:
            train_data=npy_dict['train_crime_data'][npy_dict['train_indices']]
            valid_data=npy_dict['train_crime_data'][npy_dict['valid_indices']]

            train_target=npy_dict['crime_targets'][npy_dict['train_indices']]
            valid_target=npy_dict['crime_targets'][npy_dict['valid_indices']]

        elif 'priority' in configs['mode']:
            train_data=npy_dict['train_priority_data'][npy_dict['train_indices']]
            valid_data=npy_dict['train_priority_data'][npy_dict['valid_indices']]

            train_target=npy_dict['priority_targets'][npy_dict['train_indices']]
            valid_target=npy_dict['priority_targets'][npy_dict['valid_indices']]

        return train_data, train_target, valid_data, valid_target
        

def load_dataloader(data_path,configs):
    sampler=None
    if sampler is None:
        shuffle=True
    else:
        shuffle=False
    if 'xgboost' not in configs['mode']:
        print('-------------load_dataloader--------------')
        
        train_dataset,valid_dataset=load_dataset(data_path,configs)
        train_dataloader=DataLoader(train_dataset,batch_size=configs['batch_size'],num_workers=configs['num_workers'], sampler = sampler,shuffle=shuffle)
        valid_dataloader=DataLoader(valid_dataset,batch_size=configs['batch_size'],num_workers=configs['num_workers'], shuffle = False)
        return train_dataloader,valid_dataloader
    else: #xgboost
        train_data,train_target,valid_data,valid_target = load_dataset(data_path,configs)       
        return train_data,train_target,valid_data,valid_target


In [8]:
# 우범여부와 핵심적발을 학습하는 딥러닝 모델입니다.
# 우범여부를 학습하기 위한 모델인 CrimeModel과 핵심적발의 학습을 위한 모델인 PriorityModel이 존재합니다.
# MixedModel은 앞의 CrimeModel과 PriorityModel을 통합한 모델로써, CrimeModel을 이용하여 우범여부를 학습 및 추론하고, 그 결과 및 PriorityModel을 이용하여 학심적발을 학습 및 추론합니다.

class CrimeModel(nn.Module):
    def __init__(self,input_space,output_space,configs):
        super(CrimeModel,self).__init__()
        self.model=nn.Sequential(
            nn.Linear(input_space,5000),
            nn.BatchNorm1d(5000),#5000),
            nn.ReLU(),
            nn.Linear(5000,1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000,300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(300,output_space)
        )
        self.weight1 = torch.tensor([0.2988, 1.0])
        self.criterion=nn.CrossEntropyLoss(weight=self.weight1)
        self.optimizer=torch.optim.Adam(self.model.parameters(),lr=configs['lr'],weight_decay=configs['weight_decay'])
        self.scheduler=torch.optim.lr_scheduler.StepLR(optimizer=self.optimizer,step_size=configs['lr_decay'], gamma=configs['lr_decay_rate'])

        for m in self.modules():
            if isinstance(m,(nn.Linear)):
                nn.init.kaiming_uniform_(m.weight,nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    def forward(self,x):
        y=self.model(x)
        return y


    def save_model(self,epoch,score_dict):
        dict_model={
            'epoch':epoch,
            'crime_model_state_dict':self.model.state_dict(),
        }
        dict_model.update(score_dict)
        return dict_model

    def load_model(self,dict_model):
        self.model.load_state_dict(dict_model['crime_model_state_dict'])

class PriorityModel(nn.Module):
    def __init__(self,input_space,output_space,configs):
        super(PriorityModel,self).__init__()
        self.model=nn.Sequential(
            nn.Linear(input_space,5000),
            nn.BatchNorm1d(5000),#5000),
            nn.ReLU(),
            nn.Linear(5000,1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000,300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(300,output_space)
        )
        self.weight2 = torch.tensor([1.0, 0.9431])
        self.criterion=nn.CrossEntropyLoss(weight=self.weight2)
        self.optimizer=torch.optim.Adam(self.model.parameters(),lr=configs['lr'],weight_decay=configs['weight_decay'])
        self.scheduler=torch.optim.lr_scheduler.StepLR(optimizer=self.optimizer,step_size=configs['lr_decay'], gamma=configs['lr_decay_rate'])

        for m in self.modules():
            if isinstance(m,(nn.Linear)):
                nn.init.kaiming_uniform_(m.weight,nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    def forward(self,x):
        y=self.model(x)
        return y

    def save_model(self,epoch,score_dict):
        dict_model={
            'epoch':epoch,
            'priority_model_state_dict':self.model.state_dict(),
        }
        dict_model.update(score_dict)
        return dict_model

    def load_model(self,dict_model):
        self.load_state_dict(dict_model['priority_model_state_dict'])

## Mixed Model ##

class MixedScheduler():
    def __init__(self,crime_scheduler,priority_scheduler):
        self.crime_scheduler=crime_scheduler
        self.priority_scheduler=priority_scheduler

    def step(self):
        self.crime_scheduler.step()
        self.priority_scheduler.step()

class MixedOptimizer():
    def __init__(self,crime_optim,priority_optim):
        self.crime_optim=crime_optim
        self.priority_optim=priority_optim
        self.param_groups=self.crime_optim.param_groups
    
    def step(self):
        self.crime_optim.step()
        self.priority_optim.step()

    def zero_grad(self):
        self.priority_optim.zero_grad()
        self.crime_optim.zero_grad()

class MixedLossFunction():
    def __init__(self,crime_criterion,priority_criterion,configs):
        self.crime_criterion=crime_criterion
        self.priority_criterion=priority_criterion
        self.configs=configs
        if self.configs['custom_loss']=='kd_loss':
            self.custom_criterion=KDRegLoss(configs)
        elif self.configs['custom_loss']=='fbeta_loss':
            self.custom_criterion=FBetaLoss(configs)
        else: 
            self.custom_criterion=None
    
    def __call__(self,crime_y_pred,priority_y_pred,crime_y_truth,priority_y_truth):
        crime_custom_loss=0.0
        priority_custom_loss=0.0

        crime_loss=self.crime_criterion(crime_y_pred,crime_y_truth)
        if self.custom_criterion is not None:
            crime_custom_loss=self.custom_criterion(crime_y_pred,crime_y_truth)
        crime_loss+=crime_custom_loss
        
        # search only
        idx=torch.logical_or(priority_y_truth==1,priority_y_truth==2)
        priority_y_truth=priority_y_truth[idx]-1
        priority_y_pred=priority_y_pred[torch.stack((idx,idx),dim=1)].view(-1,2)
        priority_loss=torch.nan_to_num(self.priority_criterion(priority_y_pred,priority_y_truth))
        if self.custom_criterion is not None:
            priority_custom_loss=torch.nan_to_num(self.custom_criterion(priority_y_pred,priority_y_truth))
        priority_loss+=priority_custom_loss

        return crime_loss,priority_loss

class MixedModel:
    def __init__(self,input_space,output_space,configs):
        self.crime_model=CrimeModel(input_space[0],output_space,configs)
        self.priority_model=PriorityModel(input_space[1],output_space,configs)#
        self.criterion=MixedLossFunction(self.crime_model.criterion,self.priority_model.criterion,configs)
        self.optimizer=MixedOptimizer(self.crime_model.optimizer,self.priority_model.optimizer)
        self.scheduler=MixedScheduler(self.crime_model.scheduler,self.priority_model.scheduler)
    
    def __call__(self,crime_x,priority_x):
        crime_output,priority_output=self.forward(crime_x,priority_x)
        return crime_output,priority_output
        

    def forward(self,crime_x,priority_x):
        crime_output=self.crime_model(crime_x)
        softened_crime_output=f.softmax(crime_output,dim=1).detach().clone()
        priority_output=self.priority_model(priority_x)
        return crime_output,priority_output

    def save_model(self,epoch,score_dict):
        dict_model={
            'epoch':epoch,
            'crime_model_state_dict':self.crime_model.state_dict(),
            'priority_model_state_dict':self.priority_model.state_dict(),
        }
        dict_model.update(score_dict)
        return dict_model

    def load_model(self, dict_model):
        self.crime_model.load_state_dict(dict_model['crime_model_state_dict'])
        self.priority_model.load_state_dict(dict_model['priority_model_state_dict'])
    
    def to(self,device):
        self.crime_model.to(device)
        self.priority_model.to(device)
    
    def train(self):
        self.crime_model.train()
        self.priority_model.train()

    def eval(self):
        self.crime_model.eval()
        self.priority_model.eval()

MODEL={
    'crime':CrimeModel,
    'priority':PriorityModel,
    'mixed':MixedModel,
}


In [9]:
# 학습 성능을 개선하기 위한 Custom loss에 대한 Class입니다.
# 최종 제출본을 학습한 경우에는 FBetaLoss가 사용되었습니다.

class KDRegLoss():
    def __init__(self,configs):
        self.configs=configs
        self.T=self.configs['temperature']# 20
        self.alpha=self.configs['alpha']# 0.6
    def __call__(self,outputs,labels):

        """
        loss function for mannually-designed regularization: Tf-KD_{reg}
        """
        self.alpha = 0.6
        self.T = 20.0
        correct_prob = 0.99    # the probability for correct class in u(k)
        K = outputs.size(1)

        teacher_soft = torch.ones_like(outputs)
        teacher_soft = teacher_soft*(1-correct_prob)/(K-1)  # p^d(k)
        for i in range(outputs.shape[0]):
            teacher_soft[i ,labels[i]] = correct_prob
        loss_soft_regu = nn.KLDivLoss(reduction='batchmean')(f.log_softmax(outputs, dim=1), f.softmax(teacher_soft/self.T, dim=1))*100

        KD_loss = self.alpha*loss_soft_regu

        return KD_loss

class FBetaLoss():
    def __init__(self,configs):
        self.configs=configs
        self.gae=self.configs['lambda']# 1.0
        self.beta=self.configs['beta']# 10

    def __call__(self,y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        epsilon=1e-7
        y_true = f.one_hot(y_true, 2).to(torch.float32)
        y_pred = f.softmax(y_pred, dim=1)
        
        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32) 

        precision = tp / (tp + fp + epsilon)
        recall = tp / (tp + fn + epsilon)

        f1 = (1+self.beta**2)* (precision*recall) / (self.beta**2*precision + recall + 1e-7)
        f1 = f1.clamp(min=epsilon, max=1-epsilon)
        return self.gae*(1 - f1.mean())


In [10]:
# 모델의 학습 및 성능 추론을 위한 기능을 하는 Class입니다.
# 기본 BaseLearner Class를 상속받아 CrimeLearner와 PriorityLearner를 사용합니다.

class BaseLearner:
    def __init__(self,logger,time_data, data_path, save_path, device, configs):
        self.data_path=data_path # data load
        self.save_path=save_path # save (tensorboard)
        self.device=device # cuda
        self.configs=configs
        self.logger=logger
        self.time_data=time_data

class TorchLearner(BaseLearner):
    def __init__(self, logger,time_data, data_path, save_path, device, configs):
        super(TorchLearner,self).__init__(logger,time_data, data_path, save_path, device, configs)
        self.train_dataloader,self.test_dataloader=load_dataloader(self.data_path,configs) # dataloader output(tensor) -> .numpy()
        if 'crime' in configs['mode']:
            self.input_space=self.train_dataloader.dataset[0][0].size()[0]
            self.output_space=2
        elif 'priority' in configs['mode']:
            self.input_space=self.train_dataloader.dataset[0][0].size()[0]
            self.output_space=2
        else: #mixed
            self.input_space=[self.train_dataloader.dataset[0][0].size()[0], self.train_dataloader.dataset[0][1].size()[0]]
            self.output_space=2
        self.model=MODEL[configs['mode'].split('_')[1]](self.input_space,self.output_space,configs)
        self.criterion=self.model.criterion
        self.optimizer=self.model.optimizer
        self.scheduler=self.model.scheduler
        self.logWriter=SummaryWriter(os.path.join(self.save_path,time_data))
        self.score_dict={
        'accuracy':0.0,
        'total':0.0,
        'precision':0.0,
        'recall':0.0,
        'f1score':0.0,
        'loss':0.0,
        'custom_loss':0.0,
        }
        self.best_f1score=0.0
        self.best_acc=0.0
        self.metric=dict()
        if self.configs['custom_loss']=='kd_loss':
            self.custom_criterion=KDRegLoss(configs)
        elif self.configs['custom_loss']=='fbeta_loss':
            self.custom_criterion=FBetaLoss(configs)
        else: 
            self.custom_criterion=None

    def run(self):
        self.model.to(self.configs['device'])
        self.logger.info(self.configs)

        for epoch in range(1,self.configs['epochs']+1):
            train_score_dict=copy.deepcopy(self.score_dict)
            eval_score_dict=copy.deepcopy(self.score_dict)
            #Init

            #Train
            self.metric=dict()# metric
            print('='*30)
            train_tik=time.time()
            train_score_dict=self._train(epoch,train_score_dict)
            train_tok=time.time()
            print('\n Learning Rate: {:.8f} Learning Time: {:.3f}s'.format(self.optimizer.param_groups[0]['lr'],train_tok-train_tik))
            self._epoch_end_logger(epoch,train_score_dict,'train')

            #Eval
            self.metric=dict()
            eval_score_dict=self._eval(epoch,eval_score_dict)
            self._epoch_end_logger(epoch,eval_score_dict,'eval')

            self.scheduler.step()
        
        self.logger = logging.getLogger('best')
        self.logger.info('[Mode {}] [Best Acc {:.2f}] [Best F1 {:.3f}]'.format(self.configs['mode'],self.best_acc,self.best_f1score))

        print('==End==')

    def _train(self,epoch,score_dict):
        self.model.train()
        train_loss=0.0
        train_custom_loss=0.0
        custom_loss=None

        for batch_idx, (data,targets) in enumerate(self.train_dataloader):
            data,targets=data.to(self.configs['device']),targets.to(self.configs['device'])

            outputs=self.model(data)
            if self.custom_criterion is None:
                loss=self.criterion(outputs,targets)
            else:
                loss=self.criterion(outputs,targets)
                custom_loss=self.custom_criterion(outputs,targets)
                train_custom_loss+=custom_loss.item()
                loss+=custom_loss

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            train_loss+=loss.item()
            # predictions=torch.round(outputs).view(-1)#linear regression
            predictions=torch.max(outputs,dim=1)[1].clone()
            if self.metric == dict():
                self.metric['predictions']=predictions
                self.metric['targets']=targets
            else:
                self.metric['predictions']=torch.cat((self.metric['predictions'],predictions),dim=0)
                self.metric['targets']=torch.cat((self.metric['targets'],targets),dim=0)
            if batch_idx%50==1:
                acc=(self.metric['predictions']==self.metric['targets']).sum()/self.metric['targets'].size(0)*100.0
                print('\r{}epoch {}/{}, [Acc] {:.2f} [Loss] {:.5f}'.format(epoch,self.metric['targets'].size(0),
                len(self.train_dataloader.dataset),acc,train_loss/(batch_idx+1)),end='')
            
        score_dict=self._get_score(self.metric['predictions'],self.metric['targets'],score_dict)
        score_dict['loss']=train_loss/(batch_idx+1)
        if custom_loss is not None:
            score_dict['custom_loss']=train_custom_loss/(batch_idx+1)
        return score_dict

  
    def _eval(self,epoch,score_dict):
        self.model.eval()
        eval_loss=0.0
        eval_custom_loss=0.0
        custom_loss=None

        with torch.no_grad():
            for batch_idx,(data,targets) in enumerate(self.test_dataloader):
                data,targets=data.to(self.configs['device']),targets.to(self.configs['device'])

                outputs=self.model(data)
                predictions=torch.max(outputs,dim=1)[1].clone() # cross-entropy
                # predictions=torch.round(outputs).view(-1)#linear regression
                if self.custom_criterion is None:
                    loss=self.criterion(outputs,targets)
                else:
                    loss=self.criterion(outputs,targets)
                    custom_loss=self.custom_criterion(outputs,targets)
                    eval_custom_loss+=custom_loss.item()
                    loss+=custom_loss

                eval_loss +=loss.item()
                if self.metric == dict():
                    self.metric['predictions']=predictions
                    self.metric['targets']=targets
                else:
                    self.metric['predictions']=torch.cat((self.metric['predictions'],predictions),dim=0)
                    self.metric['targets']=torch.cat((self.metric['targets'],targets),dim=0)
        score_dict=self._get_score(self.metric['predictions'],self.metric['targets'],score_dict)
        score_dict['loss']=eval_loss/(batch_idx+1)
        if custom_loss is not None:
            score_dict['custom_loss']=eval_custom_loss/(batch_idx+1)

        return score_dict

    def save_models(self,epoch, score_dict,model_type):
        dict_model= self.model.save_model(epoch,score_dict)
        torch.save(dict_model,
        os.path.join(self.save_path,self.time_data,'best_{}_model.pt'.format(model_type)))

    def load_model(self):
        dict_model=torch.load(self.save_path,self.configs['file_name'],'best_{}_model.pt'.format(self.configs['mode'].split('_')[1]))
        self.model.load_model(dict_model)

    def _get_score(self,predictions,targets,score_dict):
        score_dict=calc_score(predictions,targets,score_dict)
        return score_dict
        
    def _epoch_end_logger(self,epoch,score_dict,mode='train'):
        if mode=='eval':
            if self.best_f1score<score_dict['f1score']:
                self.best_f1score=score_dict['f1score']
                self.best_acc=score_dict['accuracy']
                self.save_models(epoch,score_dict,self.configs['mode'].split('_')[1])
        self._write_logger(epoch,self.configs['mode'].split('_')[1],score_dict,mode)

    def _write_logger(self,epoch,model_type,score_dict,mode):
        self.logger=logging.getLogger('{}'.format(mode))
        if score_dict['custom_loss']==0.0:
            #self.logger.info('\n[{} Epoch {}] [loss] {:.5f} [acc] {:.2f} [precision] {:.2f} [recall] {:.2f} [f1score] {:.2f}'.format(
            #    epoch,model_type,score_dict['loss'], score_dict['accuracy'],score_dict['precision'],score_dict['recall'],score_dict['f1score']))
            print('{} {} is completed'.format(model_type, mode))
        else:
            self.logger.info('\n[{} Epoch {}] [ce loss] {:.5f} [custom loss] {:.5f} [acc] {:.2f} [precision] {:.2f} [recall] {:.2f} [f1score] {:.2f}'.format(
                epoch,model_type,score_dict['loss'],score_dict['custom_loss'], score_dict['accuracy'],score_dict['precision'],score_dict['recall'],score_dict['f1score']))
        self.logWriter.add_scalars('{}_{}'.format(mode,model_type),score_dict,epoch)
    

class CrimeLearner(TorchLearner):
    def __init__(self,logger,imte_data, data_path, save_path, device, configs):
        super(CrimeLearner,self).__init__(logger,imte_data, data_path, save_path, device, configs)
        
class PriorityLearner(TorchLearner):
    def __init__(self,logger,imte_data, data_path, save_path, device, configs):
        super(PriorityLearner,self).__init__(logger,imte_data, data_path, save_path, device, configs)
    


In [11]:
# 모델의 학습 및 성능 추론을 위한 기능을 하는 Class입니다.
# 기본 BaseLearner Class를 상속받아 CrimeLearner와 PriorityLearner를 사용합니다.

class MixedLearner(TorchLearner):
    def __init__(self, logger:logging,time_data, data_path, save_path, device, configs):
        super(MixedLearner,self).__init__(logger,time_data, data_path, save_path, device, configs)
        self.score_dict={
            'loss':0.0,
            'total':0.0,
            'crime':copy.deepcopy(self.score_dict),
            'priority':copy.deepcopy(self.score_dict),
            'advantage':0.0
        }
        self.best_f1score={'crime':0.0,'priority':0.0}
        self.best_acc={'crime':0.0,'priority':0.0}
        self.best_epoch=0
        self.best_advantage=0.0


    def eval_run(self):
        self.model.to(self.configs['device'])
        self.logger.info(self.configs)


        crime_dict=torch.load('C:\\anaconda3\envs\\torch\Customs_Kaggle\crime-cargo-detection\\training_data\\07-16_15-13-31\\best_crime_model.pt')
        priority_dict=torch.load('C:\\anaconda3\envs\\torch\Customs_Kaggle\crime-cargo-detection\\training_data\\07-16_15-13-31\\best_priority_model.pt')
        print("========== Performances ==========")
        #print("crime F1: {:.3f} crime Acc: {:.3f}".format(crime_dict['f1score'],crime_dict['accuracy']))
        #print("priority F1: {:.3f} priority Acc: {:.3f}".format(priority_dict['f1score'],priority_dict['accuracy']))
        print("==================================")
        dict_model={**crime_dict,**priority_dict}
        self.model.load_model(dict_model)
            
        print("Model Load Complete")
        self.model.to(self.configs['device'])

        if 'mixed' not in self.configs['mode']:
            metric=self.run_ind()
        else: #mixed
            metric=self.run_mix()
        print("Prediction Complete")

    def run_mix(self):
        self.model.eval()        
        eval_score_dict=copy.deepcopy(self.score_dict)
        print('start runmix')
        self._eval(1, eval_score_dict)
        print('finish runmix')

    def run(self):
        self.model.to(self.configs['device'])
        self.logger.info(self.configs)

        for epoch in range(1,self.configs['epochs']+1):
            train_score_dict=copy.deepcopy(self.score_dict)
            eval_score_dict=copy.deepcopy(self.score_dict)
            #Init

            #Train
            self.metric=dict()# metric
            print('='*30)
            train_tik=time.time()
            train_score_dict=self._train(epoch,train_score_dict)
            train_tok=time.time()
            print('\n Learning Rate: {:.8f} Learning Time: {:.3f}s'.format(self.optimizer.param_groups[0]['lr'],train_tok-train_tik))
            self._epoch_end_logger(epoch,train_score_dict,'train')

            #Eval
            self.metric=dict()
            eval_metric={'Current':None,'Prior Best':None}
            self.logger=logging.getLogger('Current')
            eval_metric['Current']=self._eval(epoch,eval_score_dict)
            #self.logger.info('[Current eval] Crime Acc: {:.2f} F1: {:.2f} Priority Acc: {:.2f} F1 {:.2f} [adv] {:.2f}'.format(eval_metric['Current']['crime']['accuracy'],eval_metric['Current']['crime']['f1score'],eval_metric['Current']['priority']['accuracy'],eval_metric['Current']['priority']['f1score'],eval_metric['Current']['advantage']))
            if epoch>=2:
                eval_score_dict=copy.deepcopy(self.score_dict)
                self.save_tmp_model(epoch,eval_metric['Current']['crime'],'Current')
                self.load_tmp_model('Prior Best')
                self.logger=logging.getLogger('Prior Best')
                eval_metric['Prior Best']=self._eval(epoch,eval_score_dict)
                #self.logger.info('[Prior Best eval] Crime Acc: {:.2f} F1: {:.2f} Priority Acc: {:.2f} F1 {:.2f} [adv] {:.2f}'.format(eval_metric['Prior Best']['crime']['accuracy'],eval_metric['Prior Best']['crime']['f1score'],eval_metric['Prior Best']['priority']['accuracy'],eval_metric['Prior Best']['priority']['f1score'],eval_metric['Prior Best']['advantage']))
                if eval_metric['Current']['advantage']>eval_metric['Prior Best']['advantage']:
                    print('Current Advantage is Best')
                    eval_score_dict=copy.deepcopy(eval_metric['Current'])
                    self.load_tmp_model('Current')
                    self.save_tmp_model(epoch,eval_score_dict['crime'],'Prior Best')
                else:
                    print('Prior Advantage is Best')
                    eval_score_dict=copy.deepcopy(eval_metric['Prior Best'])
            else:
                eval_score_dict=copy.deepcopy(eval_metric['Current'])

            self._epoch_end_logger(epoch,eval_score_dict,'eval')
            if epoch>=2:
                self.load_tmp_model('Current')
            self.scheduler.step()
        
        self.logger = logging.getLogger('best')
        self.logger.info('[Mode {}] [Best Crime Acc {:.2f}] [Best Crime F1 {:.3f}]'.format(self.configs['mode'],self.best_acc['crime'],self.best_f1score['crime']))
        self.logger.info('[Mode {}] [Best Priority Acc {:.2f}] [Best Priority F1 {:.3f}]'.format(self.configs['mode'],self.best_acc['priority'],self.best_f1score['priority']))
        self.logger.info('[Best Advantage Score] {}'.format(self.best_advantage))
        print('==End==')    


    def _train(self,epoch,score_dict):
        self.model.train()
        train_loss=0.0

        for batch_idx, (crime_data, priority_data, crime_targets, priority_targets) in enumerate(self.train_dataloader):
            crime_data, priority_data = crime_data.to(self.configs['device']), priority_data.to(self.configs['device'])
            crime_targets, priority_targets = crime_targets.to(self.configs['device']), priority_targets.to(self.configs['device'])
            crime_outputs,priority_outputs=self.model(crime_data, priority_data)
            crime_loss,priority_loss=self.criterion(crime_outputs,priority_outputs,crime_targets,priority_targets)
            loss=crime_loss+priority_loss

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            crime_predictions=torch.max(crime_outputs,dim=1)[1].clone()
            priority_predictions=torch.max(priority_outputs,dim=1)[1].clone()

            #loss
            score_dict['crime']['loss']+=crime_loss.item()
            score_dict['priority']['loss']+=priority_loss.item()
            priority_predictions[crime_predictions==0]=-1
            self._save_score(crime_predictions,crime_targets,priority_predictions+1,priority_targets)
            train_loss+=loss.item()
            score_dict['total']+=crime_targets.size(0)
            if batch_idx%50==1:
                crime_acc=(self.metric['crime']['predictions']==self.metric['crime']['targets']).sum()/self.metric['crime']['targets'].size(0)*100.0
                priority_acc=(self.metric['priority']['predictions']==self.metric['priority']['targets']).sum()/self.metric['priority']['targets'].size(0)*100.0
                #print('\r{}epoch {}/{}, [Crime Acc] {:.2f} [Priority Acc] {:.2f}  [Loss] {:.5f}'.format(epoch,int(score_dict['total']),
                print('\r{}epoch {}/{}'.format(epoch,int(score_dict['total']), len(self.train_dataloader.dataset)),end='')

        # crime
        score_dict['crime']=self._get_score(self.metric['crime']['predictions'],self.metric['crime']['targets'],score_dict['crime'])
        # priority
        score_dict['priority']=self._get_score(self.metric['priority']['predictions'],self.metric['priority']['targets'],score_dict['priority'])

        score_dict['loss']=train_loss/(batch_idx+1)
        score_dict['crime']['loss']=score_dict['crime']['loss']/(batch_idx+1)
        score_dict['priority']['loss']=score_dict['priority']['loss']/(batch_idx+1)
        score_dict['advantage'] = (score_dict['crime']['f1score'] + score_dict['priority']['f1score'])*0.5
        return score_dict


    def _eval(self,epoch,score_dict):

        self.model.eval()
        eval_loss=0.0

        with torch.no_grad():
            for batch_idx,(crime_data,priority_data,crime_targets,priority_targets) in enumerate(self.test_dataloader):
                crime_data,priority_data=crime_data.to(self.configs['device']),priority_data.to(self.configs['device'])
                crime_targets,priority_targets=crime_targets.to(self.configs['device']),priority_targets.to(self.configs['device'])
                
                crime_outputs,priority_outputs=self.model(crime_data,priority_data)
                crime_loss,priority_loss=self.criterion(crime_outputs,priority_outputs,crime_targets,priority_targets)
                crime_predictions=torch.max(crime_outputs,dim=1)[1].clone()
                priority_predictions=torch.max(priority_outputs,dim=1)[1].clone()

                #loss
                loss=crime_loss+priority_loss
                score_dict['crime']['loss']+=crime_loss.item()
                score_dict['priority']['loss']+=priority_loss.item()
                priority_predictions[crime_predictions==0]=-1
                self._save_score(crime_predictions,crime_targets,priority_predictions+1,priority_targets)
                eval_loss +=loss.item()
        # crime
        score_dict['crime']=self._get_score(self.metric['crime']['predictions'],self.metric['crime']['targets'],score_dict['crime'])
        # priority
        score_dict['priority']=self._get_score(self.metric['priority']['predictions'],self.metric['priority']['targets'],score_dict['priority'])
                
        score_dict['loss']=eval_loss/(batch_idx+1)
        score_dict['crime']['loss']=score_dict['crime']['loss']/(batch_idx+1)
        score_dict['priority']['loss']=score_dict['priority']['loss']/(batch_idx+1)
        score_dict['advantage'] = (score_dict['crime']['f1score'] + score_dict['priority']['f1score'])*0.5
        # print('[Crime] {0:.4f}\t [Priority] {1:.4f}\t [Total F1] {2:.4f}\t'.format(score_dict['crime']['f1score'], score_dict['priority']['f1score'], score_dict['advantage']))

        return score_dict

    def _save_score(self,crime_predictions,crime_targets,priority_predictions,priority_targets):
        if self.metric==dict():
            self.metric['crime']=dict()
            self.metric['priority']=dict()
            self.metric['crime']['predictions']= crime_predictions
            self.metric['crime']['targets']= crime_targets
            self.metric['priority']['predictions']= priority_predictions
            self.metric['priority']['targets']= priority_targets
        else:
            self.metric['crime']['predictions']=torch.cat((self.metric['crime']['predictions'],crime_predictions),dim=0)
            self.metric['crime']['targets']=torch.cat((self.metric['crime']['targets'],crime_targets),dim=0)
            self.metric['priority']['predictions']=torch.cat((self.metric['priority']['predictions'],priority_predictions),dim=0)
            self.metric['priority']['targets']=torch.cat((self.metric['priority']['targets'],priority_targets),dim=0)
            
    def _epoch_end_logger(self,epoch,score_dict,mode='train'):
        for model_type in ['crime','priority']:
            this_score_dict=score_dict[model_type]
            self._write_logger(epoch,model_type,this_score_dict,mode)

        if mode=='eval':
            score_dict['advantage']=(score_dict['crime']['f1score']+score_dict['priority']['f1score'])*0.5
            self.logger.info("Advantage Score: {:.2f} Best Advantage Score: {:.2f} Best Epoch: {}".format(score_dict['advantage'],self.best_advantage,self.best_epoch))
            if epoch==1:
                self.best_epoch=1
                self.best_f1score[model_type]=score_dict[model_type]['f1score']
                self.best_acc[model_type]=score_dict[model_type]['accuracy']
                self.best_advantage=score_dict['advantage']
                for model_type in ['crime','priority']:
                    self.save_models(epoch,score_dict[model_type],model_type)
            if self.best_advantage<(score_dict['crime']['f1score']+score_dict['priority']['f1score'])*0.5:
                self.best_advantage=(score_dict['crime']['f1score']+score_dict['priority']['f1score'])*0.5
                self.best_epoch=epoch
                for model_type in ['crime','priority']:
                    self.best_f1score[model_type]=score_dict[model_type]['f1score']
                    self.best_acc[model_type]=score_dict[model_type]['accuracy']
                    self.save_models(epoch,score_dict[model_type],model_type)
    
    def load_model(self):
        crime_dict=torch.load(self.save_path,self.configs['file_name'],'best_crime_model.pt')
        priority_dict=torch.load(self.save_path,self.configs['file_name'],'best_priority_model.pt')
        dict_model={'crime':crime_dict,'priority':priority_dict}
        print("Best Advantage: {:.2f}".format((crime_dict['f1score']+priority_dict['f1score'])*0.5))
        self.model.load_model(dict_model)
        
    def save_tmp_model(self,epoch,score_dict,current):
        self.optimizer.zero_grad()
        if current=='Prior Best':
            word='best'
        elif current=='Current':
            word='tmp'
        tmp_dict={
            'crime_model_state_dict':self.model.crime_model.state_dict()}
        save_dict={**tmp_dict,**score_dict}
        torch.save(save_dict,os.path.join(self.save_path,self.time_data,'{}_crime_model.pt'.format(word)))

    def load_tmp_model(self,current):
        self.optimizer.zero_grad()
        if current=='Prior Best':
            word='best'
        elif current=='Current':
            word='tmp'
        tmp_dict=torch.load(os.path.join(self.save_path,self.time_data,'{}_crime_model.pt'.format(word)))
        self.model.crime_model.cpu()
        self.model.crime_model.load_state_dict(tmp_dict['crime_model_state_dict'])
        self.model.crime_model.to(self.configs['device'])


In [12]:
# 데이터를 생성하고, 학습을 진행한 후에 모델 파라미터를 저장하는 Cell입니다.
# 데이터 전처리 및 학습에 사용되는 Argument에 대한 Default value는 해당 노트북의 첫번째 Cell에 선언되어 있습니다.
# 학습 결과로 만들어진 모델 파라미터가 노트북이 존재하는 경로에 생성된 training_data 폴더 내에 [학습이 진행된 시간] 폴더에 생성됩니다.

configs['preprocess']=True
learner=MixedLearner(logger, time_data, data_path, save_path, device, configs)
learner.run()

-------------load_dataloader--------------
[2021-07-21 12:18:08,441] [numexpr.utils] NumExpr defaulting to 8 threads.
(71695, 21)
calc count_ratio for only train
(89619, 21)
Generate Json complete
Before priority transform shape (89619, 23)
[21/21] Finished ProcessAfter priority transform shape (89619, 41)
Before crime transform shape (89619, 23)
[21/21] Finished ProcessAfter crime transform shape (89619, 62)
Before priority transform shape (10273, 23)
[21/21] Finished ProcessAfter priority transform shape (10273, 41)
Before crime transform shape (10273, 23)
[21/21] Finished ProcessAfter crime transform shape (10273, 62)
[2021-07-21 12:18:51,787] [main] {'mode': 'train_mixed', 'seed': 11, 'batch_size': 256, 'device': 'cuda', 'colab': False, 'num_workers': 3, 'lr': 0.001, 'weight_decay': 0.0005, 'epochs': 50, 'lr_decay': 5, 'lr_decay_rate': 0.8, 'custom_loss': 'fbeta_loss', 'preprocess': True, 'split_dataset': False, 'lambda': 0.3, 'beta': 15.0, 'split_ratio': 0.8, 'only_train': True}
1

In [13]:
# 학습이 완료된 모델 파라미터를 이용하여 test.csv에 우범여부와 핵심적발에 대한 추론을 진행, test.csv에 추론 결과를 입력하여 저장하는 Class입니다.

class RecordData:
    def __init__(self,data_path,save_path,current_path,configs):
        self.configs=configs
        self.save_path=save_path
        self.test_csv=copy.deepcopy(pd.read_csv(os.path.join(data_path,'test.csv')))
        self.npy_dict=load_dataset(data_path,configs)
        test_dataset=TensorDataset(torch.from_numpy(self.npy_dict['test_crime_data']).float(),torch.from_numpy(self.npy_dict['test_priority_data']).float())
        self.data_loader=DataLoader(test_dataset)
        self.configs=load_params(configs,current_path,configs['file_name'])
        self.configs['file_name']=configs['file_name']
        if 'crime' in configs['mode']:
            self.input_space=self.data_loader.dataset[0][0].size()[0]
            self.output_space=2
        elif 'priority' in configs['mode']:
            self.input_space=self.data_loader.dataset[0][0].size()[0]
            self.output_space=2
        else: #mixed
            self.input_space=[self.data_loader.dataset[0][0].size()[0], self.data_loader.dataset[0][1].size()[0]]
            self.output_space=2
        self.model=MODEL[self.configs['mode'].split('_')[1]](self.input_space,self.output_space,configs)
        self.metric={}
        self.scoring_metric={}
    
    def _eval_before_save(self):
        mixed_dataset=TensorDataset(torch.from_numpy(self.npy_dict['train_crime_data']).float(),torch.from_numpy(self.npy_dict['train_priority_data']).float(),torch.from_numpy(self.npy_dict['crime_targets']).long(),torch.from_numpy(self.npy_dict['priority_targets']).long())
        # train_dataset=Subset(mixed_dataset,self.npy_dict['train_indices'])
        valid_dataset=Subset(mixed_dataset,self.npy_dict['valid_indices'])
        valid_dataloader=DataLoader(valid_dataset,batch_size=self.configs['batch_size'])
        self.model.eval()
        eval_loss=0.0
        score_dict={
            'loss':0.0,
            'total':0.0,
            'crime':{
            'accuracy':0.0,
            'total':0.0,
            'precision':0.0,
            'recall':0.0,
            'f1score':0.0,
            'loss':0.0,
            'custom_loss':0.0,
        },
            'priority':{
            'accuracy':0.0,
            'total':0.0,
            'precision':0.0,
            'recall':0.0,
            'f1score':0.0,
            'loss':0.0,
            'custom_loss':0.0,
        },
            'advantage':0.0
        }
        print("Start Eval")
        self.criterion=self.model.criterion
        with torch.no_grad():
            for batch_idx,(crime_data,priority_data,crime_targets,priority_targets) in enumerate(valid_dataloader):
                crime_data,priority_data=crime_data.to(self.configs['device']),priority_data.to(self.configs['device'])
                crime_targets,priority_targets=crime_targets.to(self.configs['device']),priority_targets.to(self.configs['device'])
                
                crime_outputs,priority_outputs=self.model(crime_data,priority_data)
                crime_loss,priority_loss=self.criterion(crime_outputs,priority_outputs,crime_targets,priority_targets)
                crime_predictions=torch.max(crime_outputs,dim=1)[1].clone()
                priority_predictions=torch.max(priority_outputs,dim=1)[1].clone()

                #loss
                loss=crime_loss+priority_loss
                score_dict['crime']['loss']+=crime_loss.item()
                score_dict['priority']['loss']+=priority_loss.item()
                priority_predictions[crime_predictions==0]=-1
                self._save_scoring(crime_predictions,crime_targets,priority_predictions+1,priority_targets)
                eval_loss +=loss.item()
        # crime
        score_dict['crime']=self._get_score(self.scoring_metric['crime']['predictions'],self.scoring_metric['crime']['targets'],score_dict['crime'])
        # priority
        score_dict['priority']=self._get_score(self.scoring_metric['priority']['predictions'],self.scoring_metric['priority']['targets'],score_dict['priority'])
                
        score_dict['loss']=eval_loss/(batch_idx+1)
        score_dict['crime']['loss']=score_dict['crime']['loss']/(batch_idx+1)
        score_dict['priority']['loss']=score_dict['priority']['loss']/(batch_idx+1)
        score_dict['advantage'] = (score_dict['crime']['f1score'] + score_dict['priority']['f1score'])*0.5
        print('[Crime Acc] {0:.4f}\t [Priority Acc] {1:.4f}\t'.format(score_dict['crime']['accuracy'], score_dict['priority']['accuracy']))
        print('[Crime F1] {0:.4f}\t [Priority F1] {1:.4f}\t [Advantage] {2:.4f}\t'.format(score_dict['crime']['f1score'], score_dict['priority']['f1score'], score_dict['advantage']))
        print("End Eval")

    def run(self):
        self._load_model()
        print("Model Load Complete")
        self.model.to(self.configs['device'])
        # eval하는거 성능측정 확인
        #self._eval_before_save()

        if 'mixed' not in self.configs['mode']:
            metric=self.run_ind()
        else: #mixed
            metric=self.run_mix()
        print("Prediction Complete")
        
        self._record(metric)
        print("Record Complete")
        
    def run_mix(self):
        self.model.eval()        
        metric=dict()
        with torch.no_grad():
            for batch_idx,(crime_data,priority_data) in enumerate(self.data_loader):
                crime_data=crime_data.to(self.configs['device'])
                priority_data=priority_data.to(self.configs['device'])

                crime_outputs,priority_outputs=self.model(crime_data,priority_data)
                crime_predictions=torch.max(crime_outputs,dim=1)[1].clone()
                priority_predictions=torch.max(priority_outputs,dim=1)[1].clone()

                priority_predictions[crime_predictions==0]=-1
                metric=self._save_score(crime_predictions,priority_predictions+1,metric)
        return copy.deepcopy(metric)

    def run_ind(self):
        self.model.eval()

        with torch.no_grad():
            for batch_idx,data in enumerate(self.data_loader):
                data=data.to(self.configs['device'])

                outputs=self.model(data)
                predictions=torch.max(outputs,dim=1)[1].clone() # cross-entropy
                # predictions=torch.round(outputs).view(-1)#linear regression

                if self.metric == dict():
                    self.metric['predictions']=predictions
                else:
                    self.metric['predictions']=torch.cat((self.metric['predictions'],predictions),dim=0)
        return copy.deepcopy(self.metric)

    def _record(self,metric):
        for key in self.test_csv.columns:
            if key not in ['우범여부','핵심적발','신고번호']:
                self.test_csv.drop(key,axis=1,inplace=True)
        if self.configs['mode'].split('_')[1]=='mixed':
            self.test_csv['우범여부']=metric['crime']['predictions'].cpu()
            self.test_csv['핵심적발']=metric['priority']['predictions'].cpu()
        elif self.configs['mode'].split('_')[1]=='crime':
            self.test_csv['우범여부']=metric['predictions'].cpu()
        elif self.configs['mode'].split('_')[1]=='priority':
            self.test_csv['핵심적발']=metric['predictions'].cpu()
        print('Checking length of test.csv :', len(self.test_csv.index))
        self.test_csv.to_csv(os.path.join(self.save_path,self.configs['file_name']+'_test.csv'), index = False)
        return
        
    def _load_model(self):
        if 'mixed' not in self.configs['mode']:
            dict_model=torch.load(os.path.join(self.save_path,self.configs['file_name'],'best_{}_model.pt').format(self.configs['mode'].split('_')[1]))
            self.model.load_model(dict_model)
        else: 
            crime_dict=copy.deepcopy(torch.load(os.path.join(self.save_path,self.configs['file_name'],'best_crime_model.pt')))
            priority_dict=copy.deepcopy(torch.load(os.path.join(self.save_path,self.configs['file_name'],'best_priority_model.pt')))
            # print("========== Performances ==========")
            # print("crime F1: {:.3f} crime Acc: {:.3f}".format(crime_dict['f1score'],crime_dict['accuracy']))
            # print("priority F1: {:.3f} priority Acc: {:.3f}".format(priority_dict['f1score'],priority_dict['accuracy']))
            # print("Advantage: {:.3f}".format((crime_dict['f1score']+priority_dict['f1score'])*0.5))
            # print("==================================")
            
            # Modified
            dict_model={**crime_dict,**priority_dict}
            self.model.load_model(dict_model)

            # Origin
            # self.model.load_model(crime_dict,priority_dict)
        

    def save_models(self,epoch, score_dict):
        dict_model={
            'epoch':epoch,
            '{}_model_state_dict'.format(self.configs['mode'].split('_')[1]):self.model.state_dict(),
        }.update(score_dict)
        torch.save(dict_model,
        os.path.join(self.save_path,self.time_data,'best_model.pt'))
    
    def _save_score(self,crime_predictions,priority_predictions,metric):
        if metric==dict():
            metric['crime']=dict()
            metric['priority']=dict()
            metric['crime']['predictions']= crime_predictions
            metric['priority']['predictions']= priority_predictions
        else:
            metric['crime']['predictions']=torch.cat((metric['crime']['predictions'],crime_predictions),dim=0)
            metric['priority']['predictions']=torch.cat((metric['priority']['predictions'],priority_predictions),dim=0)

        return metric
    
    def _save_scoring(self,crime_predictions,crime_targets,priority_predictions,priority_targets):
        if self.scoring_metric==dict():
            self.scoring_metric['crime']=dict()
            self.scoring_metric['priority']=dict()
            self.scoring_metric['crime']['predictions']= crime_predictions
            self.scoring_metric['crime']['targets']= crime_targets
            self.scoring_metric['priority']['predictions']= priority_predictions
            self.scoring_metric['priority']['targets']= priority_targets
        else:
            self.scoring_metric['crime']['predictions']=torch.cat((self.scoring_metric['crime']['predictions'],crime_predictions),dim=0)
            self.scoring_metric['crime']['targets']=torch.cat((self.scoring_metric['crime']['targets'],crime_targets),dim=0)
            self.scoring_metric['priority']['predictions']=torch.cat((self.scoring_metric['priority']['predictions'],priority_predictions),dim=0)
            self.scoring_metric['priority']['targets']=torch.cat((self.scoring_metric['priority']['targets'],priority_targets),dim=0)

        return self.metric
    
    def _save_scoring(self,crime_predictions,crime_targets,priority_predictions,priority_targets):
        if self.scoring_metric==dict():
            self.scoring_metric['crime']=dict()
            self.scoring_metric['priority']=dict()
            self.scoring_metric['crime']['predictions']= crime_predictions
            self.scoring_metric['crime']['targets']= crime_targets
            self.scoring_metric['priority']['predictions']= priority_predictions
            self.scoring_metric['priority']['targets']= priority_targets
        else:
            self.scoring_metric['crime']['predictions']=torch.cat((self.scoring_metric['crime']['predictions'],crime_predictions),dim=0)
            self.scoring_metric['crime']['targets']=torch.cat((self.scoring_metric['crime']['targets'],crime_targets),dim=0)
            self.scoring_metric['priority']['predictions']=torch.cat((self.scoring_metric['priority']['predictions'],priority_predictions),dim=0)
            self.scoring_metric['priority']['targets']=torch.cat((self.scoring_metric['priority']['targets'],priority_targets),dim=0)

    def _get_score(self,predictions,targets,score_dict):
        from Utils.calc_score import calc_score
        score_dict=calc_score(predictions,targets,score_dict)
        return score_dict

In [14]:
# 위의 Class를 실행하여, 학습된 모델 파라미터를 이용해 test.csv에 우범여부 및 핵심우범에 대한 추론 결과를 기록하여 저장하는 Cell입니다.
# 해당 Cell의 실행 결과로 노트북이 존재하는 경로에 training_data 폴더 내에 [학습이 진행된 시간]_test.csv의 제출 결과물이 생성됩니다.

configs['preprocess']=False
configs['file_name']=time_data
configs['mode']='record'
runner=RecordData(data_path,save_path,current_path,configs)
runner.run()

Model Load Complete
Prediction Complete
Checking length of test.csv : 10273
Record Complete
